# ShapeletTransformClassifier
Shapelets are short, discriminative subsequences. A shapelet-based classifier learns a set of these motifs and
classifies a series by its distance to each shapelet.

## Shapelet idea
For a shapelet $S$ of length $L$ and a series $X$, the distance is

$$d(S, X) = \min_{t} \left\| S - X_{t:t+L-1} \right\|_2$$

Each distance becomes a feature; a linear model or tree can then classify the series.

## Visual intuition with a toy motif

In [ ]:
import numpy as np
import plotly.graph_objects as go

rng = np.random.default_rng(7)
t = np.arange(0, 200)
y = rng.normal(0, 0.2, size=t.size)

# Inject a motif (shapelet) around t=80..99
motif = 1.8 * np.hanning(20)
y[80:100] += motif
shapelet = y[80:100]

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=y, name="series"))
fig.add_trace(go.Scatter(x=np.arange(80, 100), y=shapelet, name="shapelet", line=dict(color="#d62728", width=3)))
fig.update_layout(title="Toy series with an embedded shapelet")
fig.show()

## Distance profile
Slide the shapelet across the series and compute the distance at each position.

In [ ]:
import numpy as np
import plotly.graph_objects as go

L = len(shapelet)
distances = []
positions = []
for start in range(0, len(y) - L + 1):
    window = y[start:start + L]
    distances.append(np.linalg.norm(window - shapelet))
    positions.append(start)

fig = go.Figure()
fig.add_trace(go.Scatter(x=positions, y=distances, name="distance"))
fig.update_layout(title="Shapelet distance profile", xaxis_title="start index", yaxis_title="distance")
fig.show()

## sktime example

In [ ]:
from sktime.datasets import load_unit_test
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sklearn.metrics import classification_report

X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True)

clf = ShapeletTransformClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))

## When to use
- You expect **localized motifs** to distinguish classes.
- Interpretability matters: learned shapelets can be inspected.
- You can afford the extra computation from shapelet search.